In [1]:
from functools import partial, reduce
import os
from operator import or_, and_, contains

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from dateutil.parser import parse
import django
from django.db.models import Q
from flask_caching import Cache
#from jupyter_plotly_dash import JupyterDash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from toolz import keyfilter, merge, isiterable, get_in

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy
# but honestly it sort of isn't very good anyway

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.models import *
from plotter.graph import *
# from plotter.views import *
# from plotter.forms import *
from utils import (
    rows, 
    columns, 
    eta_methods, 
    qlist,
    keygrab, 
    in_me, 
    particular_fields_search,
    pickitems,
    pickctx
)


In [2]:
app = dash.Dash()
CACHE_CONFIG = {
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': './.cache'
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)

# we're using this to create thread-safe global values.
# also caching relatively expensive things like database lookups to the filesystem.
# is this actually slower? maybe. if so we will supplement with storing
# data in divs.
# we could instead just memoize to memory using lru_cache or similar

cset = cache_set(cache)
cget = cache_get(cache)


In [3]:
fig = main_graph()
graph_function = scatter
spec_model = MSpec 
obs_model = MObs
# update this -- a buncha fields, both spec and obs
searchable_fields = ['group','formation','member']
# active queryset is explicitly stored in global cache
cset('queryset', spec_model.objects.all().prefetch_related("observation"))
x_inputs = [
    Input('filter-1-x', 'value'),
    Input('filter-2-x', 'value'),
    Input('filter-3-x', 'value'),
    Input('axis-option-x', 'value'),
]
y_inputs = [
    Input('filter-1-y', 'value'),
    Input('filter-2-y', 'value'),
    Input('filter-3-y', 'value'),
    Input('axis-option-y', 'value')
]

In [4]:
recalculate_graph

<function plotter.graph.recalculate_graph(*args, x_inputs, y_inputs, graph_function, cget, cset)>

In [5]:
# insert 'global' variables for this app into functions

recalculate_graph = partial(
    recalculate_graph, 
    x_inputs=x_inputs,
    y_inputs=y_inputs,
    cget=cget,
    cset=cset,
    graph_function = graph_function
)

update_model_field = partial(update_model_field, cget=cget)
update_queryset = partial(update_queryset, cget=cget, cset=cset)

In [6]:
# memoize functions that seem to want caching

for function in [
    handle_search, 
    particular_fields_search,
    qlist,
    make_axis
    ]:
    function = cache.memoize()(function)

In [7]:
# app layout proper

# it's a big open UX question how to gracefully mutate filters between instruments.
# (i.e., if we want to be able to display spectra from multiple instruments
# simultaneously rather than successively).
# it might be that flexible band values are actually better than explicit filter
# selection. for now we're using this, though.
app.layout = html.Div(children = [
    html.Div(children = [
        axis_value_drop('axis-option-x'),
        filter_drop(spec_model,'filter-1-x'),
        filter_drop(spec_model,'filter-3-x'),
        filter_drop(spec_model,'filter-2-x'),
    ]),
    html.Div(children = [
        axis_value_drop('axis-option-y'),
        filter_drop(spec_model,'filter-1-y'),
        filter_drop(spec_model,'filter-3-y'),
        filter_drop(spec_model,'filter-2-y'),
    ]),
    html.Div(id = 'calculation-state', style={"display":"none"}),
    html.Div(id = 'search-trigger', style={"display":"none"}),
    # will probably want to split divs off here so we can generate new search rows
    # how do we manage that in callbacks? is there a name selector?
    # yes -- see https://dash.plotly.com/pattern-matching-callbacks
    field_drop(searchable_fields, 'field-search'),
    model_options_drop(cget('queryset'), 'group', 'value-search'),
    html.Button(id='submit-search', n_clicks=0, children='Submit'),
    html.Div(children = [fig]),
    ])

In [8]:
# register functions with app i/o

for axis in ['x', 'y']:
    app.callback(
        [
            Output('filter-1-'+axis, 'style'),
            Output('filter-2-'+axis, 'style'),
            Output('filter-3-'+axis, 'style'),
        ],
        [Input('axis-option-'+axis, 'value')]
    )(change_input_visibility)

app.callback(
    Output('main-graph', 'figure'),
    # maybe later add an explicit recalc button?
    [*x_inputs, *y_inputs, Input('search-trigger', 'value')],
)(recalculate_graph)

app.callback(
    Output('value-search', 'options'),
    [Input('field-search', 'value')],
    )(update_model_field)
    
app.callback(
    Output('search-trigger', 'value'),
    [Input('submit-search', 'n_clicks')],
    [State('field-search', 'value'),
    State('value-search', 'value')]
)(update_queryset)


<function dash.dash.Dash.callback.<locals>.wrap_func.<locals>.add_context(n_clicks, field, value, *, cget=<function cache_get.<locals>.cget at 0x7ff712cf1550>, cset=<function cache_set.<locals>.cset at 0x7ff712d33b80>)>

In [9]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Jul/2020 03:04:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 03:04:32] "GET /_dash-component-suites/dash_renderer/react@16.v1_5_0m1592852348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 03:04:32] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_5_0m1592852348.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 03:04:32] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_5_0m1592852348.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 03:04:32] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_10_1m1592852348.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 03:04:32] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_5_0m1592852348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 03:04:32] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_10_1m1592852348.min.js HTTP/1.1" 200 -
127.0.0.1 - - [01/Jul/2020 0

cats [{'filter-1-x.value': None, 'filter-2-x.value': None, 'filter-3-x.value': None, 'axis-option-x.value': 'band_max', 'filter-1-y.value': None, 'filter-2-y.value': None, 'filter-3-y.value': None, 'axis-option-y.value': 'band_avg', 'search-trigger.value': None}]
comps ['filter-1-x.value', 'filter-2-x.value', 'filter-3-x.value', 'axis-option-x.value']
[{'filter-1-x.value': None, 'filter-2-x.value': None, 'filter-3-x.value': None, 'axis-option-x.value': 'band_max'}]
cats [{'filter-1-x.value': None, 'filter-2-x.value': None, 'filter-3-x.value': None, 'axis-option-x.value': 'band_max', 'filter-1-y.value': None, 'filter-2-y.value': None, 'filter-3-y.value': None, 'axis-option-y.value': 'band_avg', 'search-trigger.value': None}]
comps ['filter-1-y.value', 'filter-2-y.value', 'filter-3-y.value', 'axis-option-y.value']
[{'filter-1-y.value': None, 'filter-2-y.value': None, 'filter-3-y.value': None, 'axis-option-y.value': 'band_avg'}]


127.0.0.1 - - [01/Jul/2020 03:04:37] "POST /_dash-update-component HTTP/1.1" 204 -


cats [{'filter-1-x.value': 'L2', 'filter-2-x.value': None, 'filter-3-x.value': None, 'axis-option-x.value': 'band_max', 'filter-1-y.value': None, 'filter-2-y.value': None, 'filter-3-y.value': None, 'axis-option-y.value': 'band_avg', 'search-trigger.value': None}]
comps ['filter-1-x.value', 'filter-2-x.value', 'filter-3-x.value', 'axis-option-x.value']
[{'filter-1-x.value': 'L2', 'filter-2-x.value': None, 'filter-3-x.value': None, 'axis-option-x.value': 'band_max'}]
cats [{'filter-1-x.value': 'L2', 'filter-2-x.value': None, 'filter-3-x.value': None, 'axis-option-x.value': 'band_max', 'filter-1-y.value': None, 'filter-2-y.value': None, 'filter-3-y.value': None, 'axis-option-y.value': 'band_avg', 'search-trigger.value': None}]
comps ['filter-1-y.value', 'filter-2-y.value', 'filter-3-y.value', 'axis-option-y.value']
[{'filter-1-y.value': None, 'filter-2-y.value': None, 'filter-3-y.value': None, 'axis-option-y.value': 'band_avg'}]


...

127.0.0.1 - - [01/Jul/2020 03:04:50] "POST /_dash-update-component HTTP/1.1" 204 -


cats [{'filter-1-x.value': 'L2', 'filter-2-x.value': 'L3', 'filter-3-x.value': None, 'axis-option-x.value': 'band_max', 'filter-1-y.value': 'L2', 'filter-2-y.value': None, 'filter-3-y.value': None, 'axis-option-y.value': 'band_min', 'search-trigger.value': None}]
comps ['filter-1-x.value', 'filter-2-x.value', 'filter-3-x.value', 'axis-option-x.value']
[{'filter-1-x.value': 'L2', 'filter-2-x.value': 'L3', 'filter-3-x.value': None, 'axis-option-x.value': 'band_max'}]
cats [{'filter-1-x.value': 'L2', 'filter-2-x.value': 'L3', 'filter-3-x.value': None, 'axis-option-x.value': 'band_max', 'filter-1-y.value': 'L2', 'filter-2-y.value': None, 'filter-3-y.value': None, 'axis-option-y.value': 'band_min', 'search-trigger.value': None}]
comps ['filter-1-y.value', 'filter-2-y.value', 'filter-3-y.value', 'axis-option-y.value']
[{'filter-1-y.value': 'L2', 'filter-2-y.value': None, 'filter-3-y.value': None, 'axis-option-y.value': 'band_min'}]


127.0.0.1 - - [01/Jul/2020 03:04:55] "POST /_dash-update-component HTTP/1.1" 200 -


cats [{'filter-1-x.value': 'L2', 'filter-2-x.value': 'L3', 'filter-3-x.value': None, 'axis-option-x.value': 'band_max', 'filter-1-y.value': 'L2', 'filter-2-y.value': 'L3', 'filter-3-y.value': None, 'axis-option-y.value': 'band_min', 'search-trigger.value': None}]
comps ['filter-1-x.value', 'filter-2-x.value', 'filter-3-x.value', 'axis-option-x.value']
[{'filter-1-x.value': 'L2', 'filter-2-x.value': 'L3', 'filter-3-x.value': None, 'axis-option-x.value': 'band_max'}]
cats [{'filter-1-x.value': 'L2', 'filter-2-x.value': 'L3', 'filter-3-x.value': None, 'axis-option-x.value': 'band_max', 'filter-1-y.value': 'L2', 'filter-2-y.value': 'L3', 'filter-3-y.value': None, 'axis-option-y.value': 'band_min', 'search-trigger.value': None}]
comps ['filter-1-y.value', 'filter-2-y.value', 'filter-3-y.value', 'axis-option-y.value']
[{'filter-1-y.value': 'L2', 'filter-2-y.value': 'L3', 'filter-3-y.value': None, 'axis-option-y.value': 'band_min'}]


127.0.0.1 - - [01/Jul/2020 03:06:20] "POST /_dash-update-component HTTP/1.1" 200 -


In [10]:
app.callback_map

{'..filter-1-x.style...filter-2-x.style...filter-3-x.style..': {'inputs': [{'id': 'axis-option-x',
    'property': 'value'}],
  'state': [],
  'callback': <function plotter.graph.change_input_visibility(calc_type)>},
 '..filter-1-y.style...filter-2-y.style...filter-3-y.style..': {'inputs': [{'id': 'axis-option-y',
    'property': 'value'}],
  'state': [],
  'callback': <function plotter.graph.change_input_visibility(calc_type)>},
 'main-graph.figure': {'inputs': [{'id': 'filter-1-x', 'property': 'value'},
   {'id': 'filter-2-x', 'property': 'value'},
   {'id': 'filter-3-x', 'property': 'value'},
   {'id': 'axis-option-x', 'property': 'value'},
   {'id': 'filter-1-y', 'property': 'value'},
   {'id': 'filter-2-y', 'property': 'value'},
   {'id': 'filter-3-y', 'property': 'value'},
   {'id': 'axis-option-y', 'property': 'value'},
   {'id': 'search-trigger', 'property': 'value'}],
  'state': [],
  'callback': <function dash.dash.Dash.callback.<locals>.wrap_func.<locals>.add_context(*args, 